In [11]:
%load_ext autoreload
%autoreload 2

In [2]:
# plot the number of events by ...

In [23]:
from models import Taxonomy
import polars as pl
import altair as alt
from tqdm import tqdm

In [4]:
db = Taxonomy("events.db")

In [16]:
version_dates = [
    dict(x)["version_date"] for x in db.cursor.execute("select distinct version_date from taxonomy")
]

In [17]:
len(version_dates)

122

In [32]:
event_counts = []
for n, version_date in tqdm(enumerate(version_dates)):
    # maybe it would be faster to just do all of this in SQL ...
    rows = list(
        db.cursor.execute(
            """
        select event_name, count(*) as count
        from taxonomy
        where version_date = version_date
        group by event_name
    """
        )
    )
    for row in rows:
        event_counts.append({"version_date": version_date, **dict(row)})

    if n > 3:
        break
event_counts = pl.DataFrame(event_counts)

4it [00:11,  2.80s/it]


In [33]:
alt.Chart(event_counts.to_pandas()).mark_bar().encode(
    x="version_date", y="count", color="event_name"
)

/Users/audy/Code/science-vessel/.venv/lib/python3.12/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/audy/Code/science-vessel/.venv/lib/python3.12/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)